In [107]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime

In [134]:
data_path = './PM2.5/'
year_scope = [2017, 2018, 2019, 2020, 2021, 2022]
os.listdir(data_path)
data_name_lst = [name for name in os.listdir(data_path) if name.startswith('PM2.5')]

In [135]:
data_name_lst

['PM2.5_2011_.xlsx',
 'PM2.5_2012_.xlsx',
 'PM2.5_2013_.xlsx',
 'PM2.5_2014_.xlsx',
 'PM2.5_2015_.xlsx',
 'PM2.5_2016_.xlsx',
 'PM2.5_2017_.xlsx',
 'PM2.5_2018_.xlsx',
 'PM2.5_2019_.xlsx',
 'PM2.5_2020_.xlsx',
 'PM2.5_2021_.xlsx',
 'PM2.5_2022_.xlsx']

In [138]:
a = []
for i in year_scope:
    for j in data_name_lst:
        if str(i) in j:
            a.append(j)

In [139]:
a

['PM2.5_2017_.xlsx',
 'PM2.5_2018_.xlsx',
 'PM2.5_2019_.xlsx',
 'PM2.5_2020_.xlsx',
 'PM2.5_2021_.xlsx',
 'PM2.5_2022_.xlsx']

## Helping Functions

In [109]:
def load_PM_data(data_path):
    PM_data = {}
    data_name_lst = [name for name in os.listdir(data_path) if name.startswith('PM2.5')]
    for name in data_name_lst:
        a_df = pd.read_excel(os.path.join(data_path, name), sheet_name='PM2.5')
        PM_data[name.split('_')[1]] = a_df
    return PM_data

def load_selected_PM_data(data_path, selected_data_name):
    PM_data = {}
    data_name_lst = selected_data_name
    for name in data_name_lst:
        a_df = pd.read_excel(os.path.join(data_path, name), sheet_name='PM2.5')
        PM_data[name.split('_')[1]] = a_df
    return PM_data

In [110]:
def check_station(ref_station, variable_station):
    ref_columns = [1]*len(ref_station)
    for idx, station_name in enumerate(ref_station):
        if station_name not in variable_station:
            ref_columns[idx] = 0
    return ref_columns

def made_locate_station(ref_name, var_name_lst, data_path):
    locate_lst = []
    ref_station= pd.read_excel(os.path.join(data_path, ref_name), sheet_name='station_detail')['รหัสสถานี'].to_list()
    for idx in range(len(var_name_lst)):
        variable_station = pd.read_excel(os.path.join(data_path, var_name_lst[idx]), sheet_name='station_detail')['รหัสสถานี'].to_list()
        locate_lst.append(check_station(ref_station, variable_station))
    return np.array(locate_lst), ref_station

## Working Space

In [111]:
# 2017 - 2022
selected_data_name = data_name_lst[6:]

In [112]:
df0 = pd.read_excel(os.path.join(data_path, selected_data_name[0]), sheet_name='station_detail')
df1 = pd.read_excel(os.path.join(data_path, selected_data_name[1]), sheet_name='station_detail')

In [113]:
df1.head()

,รหัสสถานี,ชื่อสถานี,รายละเอียดจุดติดตั้งสถานี
0,05T,แขวงบางนา เขตบางนา กทม.,กรมอุตุนิยมวิทยาบางนา
1,10T,แขวงคลองจั่น เขตบางกะปิ กทม.,เคหะชุมชนคลองจั่น
2,11T,แขวงดินแดง เขตดินแดง กทม.,สนามกีฬาการเคหะชุมชนห้วยขวาง
3,59T,แขวงพญาไท เขตพญาไท กทม.,กรมประชาสัมพันธ์
4,61T,แขวงพลับพลา เขตวังทองหลาง กทม.,โรงเรียนบดินทรเดชา (สิงห์ สิงหเสนี)


In [114]:
locate_lst, ref_station = made_locate_station(selected_data_name[-1], selected_data_name, data_path)

In [115]:
df = pd.DataFrame(locate_lst, columns=ref_station)
df_year = pd.DataFrame(year_scope, columns=['years'])
df = df_year.join(df)
df = df.loc[:,~df.columns.duplicated()].copy()
df

,years,02T,05T,10T,11T,12T,59T,61T,03T,50T,...,98T,42T,43T,44T,62T,63T,78T,80T,89T,93T
0,2017,0,1,0,0,0,1,1,0,1,...,0,0,0,1,1,1,0,1,0,0
1,2018,0,1,1,1,0,1,1,1,1,...,0,1,0,1,1,1,0,1,0,0
2,2019,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,0,0
3,2020,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,0,0
4,2021,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
5,2022,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [116]:
# PM_data = load_PM_data(data_path)
PM_data = load_selected_PM_data(data_path, selected_data_name)

In [127]:
def match_year_station(PM_data, locate_df):
    PM_data_work = PM_data.copy()
    station_lst = locate_df.columns[1:]
    stations_dict = {}
    for idx_station, now_station in enumerate(station_lst):
        year_station = locate_df['years'][locate_df[now_station] == 1].to_list()
        merge_switch = 0
        for a_year in year_station:
            a_year_df = PM_data[f'{a_year}'][[now_station]].rename(columns={now_station:f'{a_year}'})
            a_year_df['Date'] = PM_data[f'{a_year}']['Date'].apply(lambda x: datetime.strftime(x, '%m_%d'))
            if merge_switch == 0:
                station_df = a_year_df
            else:
                station_df = pd.merge(station_df, a_year_df, on='Date')
            merge_switch += 1
        stations_dict[now_station] = station_df
    return stations_dict

In [128]:
stations_dict = match_year_station(PM_data, df)

In [13]:
def sort_columns(dct):
    for key, val in dct.items():
        a = dct[key]
        columns_lst = ['Date']
        num_column = []
        for col_name in a.columns:
            try:
                num_column.append(int(col_name))
            except Exception:
                continue
        num_column = np.array(sorted(num_column)[::-1]).astype(str)
        a = a[np.append(columns_lst, num_column)]
        dct[key] = a
    return dct

In [14]:
new_stations_dict = sort_columns(stations_dict)

In [16]:
def fill_na_a_station(df):
    if df.shape[1] > 2:
        for idx in range(1, len(df.columns)-1):
            df_year = df[df.columns[idx]]
            df_before_year = df[df_year.isna()][df.columns[idx+1:]]
            fill_value = df_before_year.sum(axis=1) / (~df_before_year.isna()).sum(axis=1)
            full_df_year = df_year.fillna(fill_value)
            df[df.columns[idx]] = full_df_year
    return df


def fill_na_all_station(stations_dict):
    new_stations_dict = {}
    for key, val in stations_dict.items():
        new_stations_dict[key] = fill_na_a_station(val)
    return new_stations_dict

In [17]:
new_stations_dict_2 = fill_na_all_station(new_stations_dict)

In [18]:
new_stations_dict_2

{'02T':       Date  2022  2021  2020  2019
 0    01_01  23.0  27.0  20.0   NaN
 1    01_02  24.0  32.0  26.0   NaN
 2    01_03  27.0  46.0  32.0   NaN
 3    01_04  34.0  39.0  37.0   NaN
 4    01_05  43.0  50.0  45.0   NaN
 ..     ...   ...   ...   ...   ...
 360  12_27  39.0  36.0  41.0  36.0
 361  12_28  48.0  19.0  27.0  36.0
 362  12_29  43.0  19.0  36.0  41.0
 363  12_30  27.0  23.0  36.0  37.0
 364  12_31  29.0  23.0  23.0  41.0
 
 [365 rows x 5 columns],
 '05T':       Date  2022  2021  2020  2019  2018  2017
 0    01_01  21.0    20    22  15.0  17.0  33.0
 1    01_02  24.0    25    24  21.0  25.0  28.0
 2    01_03  27.0    37    28  24.0  34.0  31.0
 3    01_04  33.0    31    35  27.0  41.0  37.0
 4    01_05  43.0    31    41  33.0  42.0  45.0
 ..     ...   ...   ...   ...   ...   ...   ...
 360  12_27  35.0    29    32  36.0  60.0  38.0
 361  12_28  43.0    17    16  37.0  65.0  18.0
 362  12_29  38.0    19    24  40.0  25.0  14.0
 363  12_30  24.0    23    21  39.0  14.0  14.0

In [69]:
def convert_to_original_yearfill(stations_dict, PM_data):
    new_PM_data = {}
    for year_key, year_df in PM_data.items():
        print('Current :', year_key)
        uniform_df = pd.DataFrame({'Date': year_df['Date'].apply(lambda x: datetime.strftime(x, '%m_%d'))})
        for station_key, station_df in stations_dict.items():
            if year_key in station_df.columns:
                station_in_year = station_df[['Date', year_key]].rename(columns={year_key:station_key}).drop_duplicates(subset=['Date'])
                uniform_df = pd.merge(uniform_df, station_in_year, on='Date')
        new_PM_data[year_key] = uniform_df
    return new_PM_data

In [76]:
new_PM_data = convert_to_original_yearfill(new_stations_dict_2, PM_data)

Current : 2017
Current : 2018
Current : 2019
Current : 2020
Current : 2021
Current : 2022


In [77]:
def save_to_csv(PM_data, save_path):
    for key, val in PM_data.items():
        val['Date'] = val['Date'].apply(lambda x: key + '_' + x)
        val.to_csv(os.path.join(save_path, f'fill_by_years.{key}.csv'), index=False)

In [78]:
# save_to_csv(new_PM_data, './Clean_Data/')